In [14]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import PassiveAggressiveRegressor
import numpy as np
from sklearn.metrics import mean_squared_error,  r2_score, mean_absolute_error

In [15]:
import warnings
warnings.filterwarnings('ignore')


In [17]:
path= '../data/bus_running_times_feature_added_all.csv'
df = pd.read_csv(path)
df = df[df['week_no'] < 35]


In [18]:
df = df[['deviceid','week_no','segment','length','direction',
 'month','day','day_of_week',
 'time_of_day',
 'dt(n-1)','rt(w-1)','rt(w-2)','rt(w-3)','rt(t-1)','rt(t-2)','rt(n-1)','rt(n-2)','rt(n-3)',
 'precip','windspeed','temp','run_time_in_seconds']]
df

,deviceid,week_no,segment,length,direction,month,day,day_of_week,time_of_day,dt(n-1),...,rt(w-3),rt(t-1),rt(t-2),rt(n-1),rt(n-2),rt(n-3),precip,windspeed,temp,run_time_in_seconds
0,262.0,1.0,1.0,0.6261,1.0,10.0,1.0,4.0,6.50,0.0,...,96.0,96.0,96.0,96.0,96.0,96.0,0.0,6.1,20.0,69.0
1,262.0,1.0,2.0,1.2808,1.0,10.0,1.0,4.0,6.50,74.0,...,247.0,247.0,247.0,69.0,247.0,247.0,0.0,6.1,20.0,210.0
2,262.0,1.0,3.0,2.1125,1.0,10.0,1.0,4.0,6.75,0.0,...,506.0,506.0,506.0,210.0,69.0,506.0,0.0,6.1,20.0,496.0
3,262.0,1.0,4.0,1.5513,1.0,10.0,1.0,4.0,6.75,6.0,...,192.0,192.0,192.0,496.0,210.0,69.0,0.0,6.1,20.0,195.0
4,262.0,1.0,5.0,0.8450,1.0,10.0,1.0,4.0,6.75,0.0,...,114.0,114.0,114.0,195.0,496.0,210.0,0.0,6.1,20.0,97.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
182524,262.0,34.0,30.0,2.5600,2.0,9.0,11.0,6.0,17.50,43.0,...,407.0,407.0,407.0,170.0,149.0,49.0,0.0,2.1,26.1,398.0
182525,262.0,34.0,31.0,0.4200,2.0,9.0,11.0,6.0,17.50,15.0,...,62.0,62.0,62.0,398.0,170.0,149.0,0.0,2.1,26.1,55.0
182526,262.0,34.0,32.0,1.3000,2.0,9.0,11.0,6.0,17.50,0.0,...,196.0,196.0,196.0,55.0,398.0,170.0,0.0,2.1,26.1,188.0
182527,262.0,34.0,33.0,1.2200,2.0,9.0,11.0,6.0,17.75,15.0,...,169.0,169.0,169.0,188.0,55.0,398.0,0.0,2.1,26.1,150.0


In [19]:
# dropping the NA data
df = df.dropna(subset=['run_time_in_seconds'])
df.isnull().sum()

deviceid               0
week_no                0
segment                0
length                 0
direction              0
month                  0
day                    0
day_of_week            0
time_of_day            0
dt(n-1)                0
rt(w-1)                0
rt(w-2)                0
rt(w-3)                0
rt(t-1)                0
rt(t-2)                0
rt(n-1)                0
rt(n-2)                0
rt(n-3)                0
precip                 0
windspeed              0
temp                   0
run_time_in_seconds    0
dtype: int64

In [25]:
from river import evaluate
from river import neural_net as nn
from river import optim
from river import preprocessing as pp
from river.metrics import MAE, MSE, R2, RMSE
from river.stream import iter_pandas
from river.metrics.base import Metrics
from river.evaluate import progressive_val_score



MLP

In [30]:
stream = iter_pandas(X=df.drop(columns=['run_time_in_seconds']), y=df['run_time_in_seconds'])

In [37]:
stream = iter_pandas(X=df.drop(columns=['run_time_in_seconds']), y=df['run_time_in_seconds'])
model = (
     pp.StandardScaler() |
     nn.MLPRegressor(
         hidden_dims=(100,50),
         activations=(
             nn.activations.ReLU,
             nn.activations.ReLU,
             nn.activations.Identity
         ),
         optimizer=optim.Adam(0.001),  # Trying a different optimizer with a different learning rate
         seed=42
     )
 )

metrics = Metrics(metrics=[MAE(),R2(),MSE(),RMSE()])

progressive_val_score(dataset=stream,
                      model=model,
                      metric=metrics,
                      print_every=9000)

[9,000] MAE: 36.365359, R2: 0.694004, MSE: 3,911.561601, RMSE: 62.542478
[18,000] MAE: 35.831918, R2: 0.75475, MSE: 3,621.860394, RMSE: 60.181894
[27,000] MAE: 35.105732, R2: 0.777739, MSE: 3,470.310333, RMSE: 58.90934
[36,000] MAE: 34.014283, R2: 0.787931, MSE: 3,279.765124, RMSE: 57.269234
[45,000] MAE: 33.706482, R2: 0.78952, MSE: 3,202.257426, RMSE: 56.588492
[54,000] MAE: 34.029582, R2: 0.78871, MSE: 3,294.348886, RMSE: 57.396419
[63,000] MAE: 33.230493, R2: 0.792289, MSE: 3,141.275306, RMSE: 56.047081
[72,000] MAE: 33.04223, R2: 0.794279, MSE: 3,147.817527, RMSE: 56.105414
[81,000] MAE: 33.474772, R2: 0.792173, MSE: 3,172.758935, RMSE: 56.327249
[90,000] MAE: 33.824948, R2: 0.791004, MSE: 3,203.578972, RMSE: 56.600168
[99,000] MAE: 34.151704, R2: 0.789781, MSE: 3,252.916355, RMSE: 57.034344
[108,000] MAE: 34.251699, R2: 0.790254, MSE: 3,235.210714, RMSE: 56.878913
[117,000] MAE: 34.500882, R2: 0.788044, MSE: 3,276.342791, RMSE: 57.239347
[126,000] MAE: 34.754894, R2: 0.785983, MS

MAE: 36.02886, R2: 0.778652, MSE: 3,532.357881, RMSE: 59.433643

ARF

In [39]:
stream = iter_pandas(X=df.drop(columns=['run_time_in_seconds']), y=df['run_time_in_seconds'])

from river.forest import ARFRegressor   
from river import preprocessing
model = (
    preprocessing.StandardScaler() |
    ARFRegressor(seed=42)
)

metrics = Metrics(metrics=[MAE(),R2(),MSE(),RMSE()])

progressive_val_score(dataset=stream,
                      model=model,
                      metric=metrics,
                      print_every=9000)

[9,000] MAE: 30.023172, R2: 0.793084, MSE: 2,645.025749, RMSE: 51.429814
[18,000] MAE: 31.265408, R2: 0.792106, MSE: 3,070.197605, RMSE: 55.409364
[27,000] MAE: 31.337101, R2: 0.794093, MSE: 3,214.962263, RMSE: 56.700637
[36,000] MAE: 30.66012, R2: 0.801758, MSE: 3,065.93225, RMSE: 55.370861
[45,000] MAE: 30.396084, R2: 0.803116, MSE: 2,995.402084, RMSE: 54.730267
[54,000] MAE: 30.869912, R2: 0.79574, MSE: 3,184.743088, RMSE: 56.433528
[63,000] MAE: 30.337112, R2: 0.798835, MSE: 3,042.287246, RMSE: 55.156933
[72,000] MAE: 30.209963, R2: 0.798628, MSE: 3,081.265676, RMSE: 55.509149
[81,000] MAE: 30.699129, R2: 0.797336, MSE: 3,093.928377, RMSE: 55.623092
[90,000] MAE: 31.106716, R2: 0.795685, MSE: 3,131.813431, RMSE: 55.962607
[99,000] MAE: 31.541819, R2: 0.793793, MSE: 3,190.825532, RMSE: 56.487393
[108,000] MAE: 31.764115, R2: 0.793834, MSE: 3,179.994122, RMSE: 56.391437
[117,000] MAE: 32.077681, R2: 0.790818, MSE: 3,233.462704, RMSE: 56.863545
[126,000] MAE: 32.397047, R2: 0.788585, 

MAE: 33.810312, R2: 0.780132, MSE: 3,508.738548, RMSE: 59.234606

KNN

In [41]:
stream = iter_pandas(X=df.drop(columns=['run_time_in_seconds']), y=df['run_time_in_seconds'])

from river.neighbors import KNNRegressor   
from river import preprocessing
model = (
    KNNRegressor(n_neighbors=5)
)

metrics = Metrics(metrics=[MAE(),R2(),MSE(),RMSE()])

progressive_val_score(dataset=stream,
                      model=model,
                      metric=metrics,
                      print_every=9000)

[9,000] MAE: 34.460637, R2: 0.741803, MSE: 3,300.545663, RMSE: 57.450376
[18,000] MAE: 36.304741, R2: 0.743303, MSE: 3,790.918067, RMSE: 61.570432
[27,000] MAE: 36.59639, R2: 0.74522, MSE: 3,978.050581, RMSE: 63.071789
[36,000] MAE: 36.347254, R2: 0.744609, MSE: 3,949.774059, RMSE: 62.847228
[45,000] MAE: 36.266719, R2: 0.742664, MSE: 3,915.129804, RMSE: 62.570998
[54,000] MAE: 36.699647, R2: 0.739441, MSE: 4,062.538737, RMSE: 63.738048
[63,000] MAE: 36.135123, R2: 0.741267, MSE: 3,912.900814, RMSE: 62.553184
[72,000] MAE: 36.210377, R2: 0.739678, MSE: 3,983.282422, RMSE: 63.113251
[81,000] MAE: 36.646374, R2: 0.737488, MSE: 4,007.586981, RMSE: 63.305505
[90,000] MAE: 37.062246, R2: 0.735373, MSE: 4,056.311683, RMSE: 63.68918
[99,000] MAE: 37.449743, R2: 0.734599, MSE: 4,106.803438, RMSE: 64.084346
[108,000] MAE: 37.684157, R2: 0.73441, MSE: 4,096.57857, RMSE: 64.00452
[117,000] MAE: 37.959817, R2: 0.731849, MSE: 4,144.986474, RMSE: 64.381569
[126,000] MAE: 38.243314, R2: 0.729294, MSE

MAE: 39.731814, R2: 0.71898, MSE: 4,484.63898, RMSE: 66.967447

HTR

In [42]:
stream = iter_pandas(X=df.drop(columns=['run_time_in_seconds']), y=df['run_time_in_seconds'])

from river import tree

model = (
    preprocessing.StandardScaler() |
    tree.HoeffdingTreeRegressor(
        grace_period=100,
        model_selector_decay=0.9
    )
)

metrics = Metrics(metrics=[MAE(),R2(),MSE(),RMSE()])

progressive_val_score(dataset=stream,
                      model=model,
                      metric=metrics,
                      print_every=9000)

[9,000] MAE: 40.917041, R2: 0.503196, MSE: 6,350.67824, RMSE: 79.691143
[18,000] MAE: 41.47804, R2: 0.614162, MSE: 5,698.085925, RMSE: 75.485667
[27,000] MAE: 41.18244, R2: 0.645464, MSE: 5,535.615065, RMSE: 74.401714
[36,000] MAE: 39.849337, R2: 0.664885, MSE: 5,182.74944, RMSE: 71.991315
[45,000] MAE: 38.970409, R2: 0.658555, MSE: 5,194.763561, RMSE: 72.074708
[54,000] MAE: 38.688035, R2: 0.670204, MSE: 5,142.050915, RMSE: 71.708095
[63,000] MAE: 37.690683, R2: 0.682713, MSE: 4,798.437674, RMSE: 69.270756
[72,000] MAE: 37.055507, R2: 0.694038, MSE: 4,681.643347, RMSE: 68.422535
[81,000] MAE: 38.328042, R2: 0.686243, MSE: 4,789.911445, RMSE: 69.209186
[90,000] MAE: 38.960978, R2: 0.685193, MSE: 4,825.480709, RMSE: 69.46568
[99,000] MAE: 39.395004, R2: 0.686055, MSE: 4,857.969897, RMSE: 69.699138
[108,000] MAE: 39.640069, R2: 0.687903, MSE: 4,813.921098, RMSE: 69.382426
[117,000] MAE: 39.818625, R2: 0.687937, MSE: 4,823.762474, RMSE: 69.453311
[126,000] MAE: 40.048374, R2: 0.68796, MSE

MAE: 40.978042, R2: 0.688103, MSE: 4,977.375901, RMSE: 70.55052

SGT Regression

In [43]:
stream = iter_pandas(X=df.drop(columns=['run_time_in_seconds']), y=df['run_time_in_seconds'])

from river import tree

model = tree.SGTRegressor(
    delta=0.01,
    lambda_value=0.01,
    grace_period=20,
    feature_quantizer=tree.splitter.DynamicQuantizer(std_prop=0.1)
)

metrics = Metrics(metrics=[MAE(),R2(),MSE(),RMSE()])

progressive_val_score(dataset=stream,
                      model=model,
                      metric=metrics,
                      print_every=9000)

[9,000] MAE: 32.757676, R2: 0.75162, MSE: 3,175.054519, RMSE: 56.347622
[18,000] MAE: 34.445237, R2: 0.741367, MSE: 3,819.513551, RMSE: 61.802213
[27,000] MAE: 34.854488, R2: 0.73669, MSE: 4,111.233486, RMSE: 64.118901
[36,000] MAE: 34.206269, R2: 0.744201, MSE: 3,956.074414, RMSE: 62.897332
[45,000] MAE: 34.093498, R2: 0.744656, MSE: 3,884.82171, RMSE: 62.328338
[54,000] MAE: 34.561596, R2: 0.738525, MSE: 4,076.82823, RMSE: 63.850045
[63,000] MAE: 33.948538, R2: 0.743217, MSE: 3,883.41156, RMSE: 62.317025
[72,000] MAE: 33.784441, R2: 0.744179, MSE: 3,914.410834, RMSE: 62.565253
[81,000] MAE: 34.485339, R2: 0.742828, MSE: 3,926.068509, RMSE: 62.658347
[90,000] MAE: 35.039452, R2: 0.740675, MSE: 3,975.03293, RMSE: 63.047862
[99,000] MAE: 35.487308, R2: 0.739676, MSE: 4,028.238921, RMSE: 63.468409
[108,000] MAE: 35.650225, R2: 0.740882, MSE: 3,996.75795, RMSE: 63.219917
[117,000] MAE: 35.892401, R2: 0.739195, MSE: 4,031.439377, RMSE: 63.493617
[126,000] MAE: 36.182131, R2: 0.737599, MSE:

MAE: 37.742652, R2: 0.726145, MSE: 4,370.295358, RMSE: 66.108209

In [6]:
def predict(model, dt, t):
    preds = model.predict(dt)
    rmse = np.sqrt(mean_squared_error(t['run_time_in_seconds'], preds)) 
    mae = mean_absolute_error(t['run_time_in_seconds'], preds)
    print("MAE (1): %f" % (mae)) 
    

<hr>

In [7]:
batch = []
dbatch =[]
batch.append(df[df['week_no'] <= 19]) 
for i in range(20,24):
    batch.append(df[df['week_no'] == i])
for i in batch:
    dbatch.append([i.drop(columns=['week_no','run_time_in_seconds']), i['run_time_in_seconds']])

In [8]:
def inc_train(is_initial, data_batch, prev_model):
    if is_initial:
        curr_model = PassiveAggressiveRegressor(C=1.0, max_iter=2000, random_state=42, epsilon=0.1, loss='squared_epsilon_insensitive')
        curr_model.fit(data_batch[0], data_batch[1])
    else:
        curr_model = prev_model.partial_fit(data_batch[0], data_batch[1])
    return curr_model

In [9]:
params = {'objective': 'reg:squarederror', 'verbose': False, 'colsample_bytree': 0.7,
    'learning_rate': 0.1,
    'max_depth': 6,
    'alpha': 10,
    'subsample':0.7,
    
    'n_estimators': 100}

In [43]:
batch[0]

,deviceid,week_no,segment,length,direction,month,day,day_of_week,time_of_day,dt(n-1),...,rt(w-3),rt(t-1),rt(t-2),rt(n-1),rt(n-2),rt(n-3),precip,windspeed,temp,run_time_in_seconds
0,262.0,1.0,1.0,0.6261,1.0,10.0,1.0,4.0,6.50,0.0,...,96.0,96.0,96.0,96.0,96.0,96.0,0.0,6.1,20.0,69.0
1,262.0,1.0,2.0,1.2808,1.0,10.0,1.0,4.0,6.50,74.0,...,247.0,247.0,247.0,69.0,247.0,247.0,0.0,6.1,20.0,210.0
2,262.0,1.0,3.0,2.1125,1.0,10.0,1.0,4.0,6.75,0.0,...,506.0,506.0,506.0,210.0,69.0,506.0,0.0,6.1,20.0,496.0
3,262.0,1.0,4.0,1.5513,1.0,10.0,1.0,4.0,6.75,6.0,...,192.0,192.0,192.0,496.0,210.0,69.0,0.0,6.1,20.0,195.0
4,262.0,1.0,5.0,0.8450,1.0,10.0,1.0,4.0,6.75,0.0,...,114.0,114.0,114.0,195.0,496.0,210.0,0.0,6.1,20.0,97.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
130455,1358.0,19.0,30.0,2.5612,2.0,2.0,6.0,6.0,17.25,143.0,...,414.0,436.0,436.0,233.0,103.0,45.0,0.0,10.4,25.7,473.0
130456,1358.0,19.0,31.0,0.4169,2.0,2.0,6.0,6.0,17.50,0.0,...,84.0,59.0,60.0,473.0,233.0,103.0,0.0,10.4,25.7,60.0
130457,1358.0,19.0,32.0,1.2960,2.0,2.0,6.0,6.0,17.50,0.0,...,266.0,223.0,245.0,60.0,473.0,233.0,0.0,10.4,25.7,281.0
130458,1358.0,19.0,33.0,1.2245,2.0,2.0,6.0,6.0,17.50,59.0,...,180.0,198.0,150.0,281.0,60.0,473.0,0.0,10.4,25.7,135.0


In [10]:
models = []
for i in range(len(batch)):
    if i!=0:
        curr_model = inc_train(False,  dbatch[i], models[i-1])
    else:
        curr_model = inc_train(True, dbatch[i], None)
    models.append(curr_model)
    for j in range(len(batch)):
        print(f"Results of data batch {j+1}  with model {i+1}")
        predict(curr_model, dbatch[j][0], batch[j])

Results of data batch 1  with model 1
MAE (1): 52.802774
Results of data batch 2  with model 1
MAE (1): 52.682691
Results of data batch 3  with model 1
MAE (1): 56.534648
Results of data batch 4  with model 1
MAE (1): 52.815582
Results of data batch 5  with model 1
MAE (1): 58.160050
Results of data batch 1  with model 2
MAE (1): 79.934264
Results of data batch 2  with model 2
MAE (1): 81.203753
Results of data batch 3  with model 2
MAE (1): 84.803804
Results of data batch 4  with model 2
MAE (1): 87.045376
Results of data batch 5  with model 2
MAE (1): 84.242652
Results of data batch 1  with model 3
MAE (1): 41.275217
Results of data batch 2  with model 3
MAE (1): 41.682451
Results of data batch 3  with model 3
MAE (1): 44.550561
Results of data batch 4  with model 3
MAE (1): 43.259581
Results of data batch 5  with model 3
MAE (1): 47.278976
Results of data batch 1  with model 4
MAE (1): 37.429830
Results of data batch 2  with model 4
MAE (1): 37.270966
Results of data batch 3  with m